# Bibliotecas

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/0 (e.g. pd.read_csv)
import re # for regex
import unicodedata
import spacy

# Funções

## Normalizador

In [2]:
def norm(s):
    """
    Normaliza o texto colocando em caixa baixa e removendo acentuação/pontuação.

    Parameters
    ----------
    s : str
        Entrada de texto.

    Returns
    -------
    str
        Texto normalizado. Se `s` não for uma string, retorna inalterado.

    Examples
    --------
    >>> norm("Árvore")
    'arvore'

    >>> norm("Fomos à escola ontem, Fernando não estava lá.")
    'fomos a escola ontem fernando nao estava la'
    """
    if not isinstance(s, str):
        return s
        
    s = s.lower()
    s = s.replace("-", " ")
    
    s = ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
  
    return re.sub(r"[^A-Za-zÀ-ÿ ]+", "", s)

# Carregar dados

In [3]:
df_class_words = pd.read_csv('../data/sentilex_v1.0.0.csv', sep = ';')

# Entrada de texto

In [4]:
text_neutral = """O evento começou pontualmente às 10h da manhã, conforme anunciado no cronograma. O palestrante principal falou por exatos 45 minutos, cobrindo os tópicos listados na descrição da palestra. Após a apresentação, os participantes foram direcionados ao salão adjacente para o intervalo. A organização informou que a próxima sessão começará em 15 minutos."""

In [5]:
text_positive = """A experiência de hoje foi absolutamente fantástica! Desde o primeiro momento, a equipe demonstrou um profissionalismo e uma simpatia incríveis, superando todas as minhas expectativas. O produto não é apenas bom, é maravilhoso e funciona perfeitamente. A qualidade é evidente em cada detalhe. Estou genuinamente impressionado e muito satisfeito com o resultado. Com certeza recomendo a todos; foi uma escolha excelente e valeu cada centavo!"""

In [6]:
text_negative = """Minha experiência hoje foi absolutamente terrível. Desde o início, o atendimento demonstrou um descaso e uma falta de profissionalismo inacreditáveis, frustrando todas as minhas expectativas. O produto não é apenas ruim, é péssimo e falha constantemente. A baixa qualidade é evidente em cada detalhe. Estou genuinamente decepcionado e muito insatisfeito com o resultado. Com certeza não recomendo a ninguém; foi uma escolha horrível e um desperdício de dinheiro."""

In [7]:
ig_text = "É bem comum as pessoas terem gostos próprios."

In [29]:
txt_test_1p = "Serviço rápido, funcionários atentos, resolução eficiente do que eu precisava."

txt_test_1ng = "Produto chegou quebrado, atendimento falhou em resolver e a experiência inteira foi desgastante."

txt_test_1nt = "A entrega ocorreu no prazo, sem surpresa boa ou ruim, exatamente o esperado."

txt_test_2p = "A atualização recente melhorou desempenho, reduziu erros e deixou o uso mais fluido."

txt_test_2ng = "O processo ficou confuso, documentação incompleta e resultados abaixo do que foi prometido."

# Tokenização

Aplica a normalização nos textos:

In [10]:
df_texts.texts = df_texts.texts.apply(norm).astype("string")
df_class_words.word = df_class_words.word.apply(norm).astype("string")

In [11]:
df_class_words

,word,polarity_n0,polarity_n1
0,a vontade,1.0,NaN
1,abafado,-1.0,NaN
2,abafante,-1.0,NaN
3,abaixado,-1.0,NaN
4,abalado,-1.0,NaN
...,...,...,...
7009,zombeteiro,-1.0,NaN
7010,zonzeira,-1.0,NaN
7011,zonzo,-1.0,NaN
7012,zote,-1.0,NaN


Tokeniza o texto:

In [18]:
nlp = spacy.load('pt_core_news_sm')

text = df_texts.texts[1]
doc = nlp(text)

# Extraindo os tokens
tokens = [token.text for token in doc]
df_text_tokens = pd.DataFrame(tokens)
df_text_tokens.columns = ['word']

In [19]:
df_text_tokens.head()

,word
0,a
1,experiencia
2,de
3,hoje
4,foi


# Mesclando dados

In [20]:
merged = df_text_tokens.merge(
    df_class_words[['word', 'polarity_n0','polarity_n1']],
    on='word',
    how='left'
)

# Removendo valores nulos

In [21]:
merged = merged.dropna(subset=['polarity_n0'])

# Contando polaridades

In [22]:
contagem = merged['polarity_n0'].value_counts().reindex([-1, 0, 1], fill_value=0)
contagem

polarity_n0
-1    0
 0    1
 1    7
Name: count, dtype: int64

# Checando o resultado

In [23]:
if (contagem == contagem.max()).sum() > 1:
    resultado = None     # empate
else:
    resultado = contagem.idxmax()

if resultado == 1:
    print("positivo")
elif resultado == -1:
    print("negativo")
else :
    print("neutro")

positivo


# Juntando tudo

In [30]:
nlp = spacy.load('pt_core_news_sm')

text = txt_test_1p
doc = nlp(text)

# Extraindo os tokens
tokens = [token.text for token in doc]
df_text_tokens = pd.DataFrame(tokens)
df_text_tokens.columns = ['word']

merged = df_text_tokens.merge(
    df_class_words[['word', 'polarity_n0','polarity_n1']],
    on='word',
    how='left'
)

merged = merged.dropna(subset=['polarity_n0'])

contagem = merged['polarity_n0'].value_counts().reindex([-1, 0, 1], fill_value=0)
contagem

if (contagem == contagem.max()).sum() > 1:
    resultado = None     # empate
else:
    resultado = contagem.idxmax()

if resultado == 1:
    print("positivo")
elif resultado == -1:
    print("negativo")
else :
    print("neutro")

positivo
